In [1]:
import time

start = time.time()

import os
import glob
from tqdm import tqdm
from random import randint

import numpy as np
import pydicom

import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.animation as anim
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec


import imageio
from skimage.transform import resize
from skimage.util import montage

from IPython.display import Image as show_gif

import warnings
warnings.simplefilter("ignore")


In [2]:
sample_id = 'F:/1_COLLEGE/TERM 11/Grad/MFG 718M/FIALS/MRI/test/00001/'
path_x = 'F:/1_COLLEGE/TERM 11/Grad/MFG 718M/FIALS/MRI/test/00001/T2w/*.dcm'
start = len('F:/1_COLLEGE/TERM 11/Grad/MFG 718M/FIALS/MRI/test/00001/T2w/Image-')
end = len('.dcm')
path_to_slices = sorted(glob.glob(path_x), key= lambda x: int(x[start:-end]))

In [3]:
class Image3dToGIF3d:
    """
    Displaying 3D images in 3d axes.
    Parameters:
        img_dim: shape of cube for resizing.
        figsize: figure size for plotting in inches.
    Step by step explanation - https://terbium.io/2017/12/matplotlib-3d/
    """
    def __init__(self, 
                 img_dim: tuple = (55, 55, 55),
                 figsize: tuple = (15, 10),
                 binary: bool = False,
                 normalizing: bool = True,
                ):
        """Initialization."""
        self.img_dim = img_dim
        print(img_dim)
        self.figsize = figsize
        self.binary = binary
        self.normalizing = normalizing

    def _explode(self, data: np.ndarray):
        """
        Takes: array and return an array twice as large in each dimension,
        with an extra space between each voxel.
        """
        shape_arr = np.array(data.shape)
        size = shape_arr[:3] * 2 - 1
        exploded = np.zeros(np.concatenate([size, shape_arr[3:]]),
                            dtype=data.dtype)
        exploded[::2, ::2, ::2] = data
        return exploded

    def _expand_coordinates(self, indices: np.ndarray):
        """ 
        Parameters:
            indices: coordinats of array with only original values
            (before explode transformaion)
        
        Returns:
        The arrays of values each dimensions (x y z) as arguments needed 
        for the plt.figure.voxels functionto extend coordinates
        for the rendering only colored voxels"""
        x, y, z = indices
        x[1::2, :, :] += 1
        y[:, 1::2, :] += 1
        z[:, :, 1::2] += 1
        return x, y, z
    
    def _normalize(self, arr: np.ndarray):
        """Normilize image value between 0 and 1."""
        arr_min = np.min(arr)
        return (arr - arr_min) / (np.max(arr) - arr_min)

    
    def _scale_by(self, arr: np.ndarray, factor: int = 2):
        """
        Scale 3d Image to factor (guesstimated transformation).
        Parameters:
            arr: 3d image for scalling.
            factor: factor for scalling.
        """
        mean = np.mean(arr)
        return (arr - mean) * factor + mean
    
    def get_transformed_data(self, data: np.ndarray):
        """Data transformation: normalization, scaling, resizing."""
        if self.binary:
            resized_data = resize(data, self.img_dim, preserve_range=True)
            return np.clip(resized_data.astype(np.uint8), 0, 1).astype(np.float32)
            
        norm_data = np.clip(self._normalize(data)-0.1, 0, 1) ** 0.4
        scaled_data = np.clip(self._scale_by(norm_data) - 0.1, 0, 1)
        resized_data = resize(scaled_data, self.img_dim, preserve_range=True)
        
        return resized_data
    
    def plot_cube(self,
                  cube,
                  title: str = '', 
                  init_angle: int = 0,
                  make_gif: bool = False,
                  path_to_save: str = 'filename.gif'
                 ):
        """
        Plot 3d data.
        -> Take array 
        -> return an array twice as large in each dimension,
        (with an extra space between each voxel)
        -> expand coordinates of each voxel for core rendering (to remove gaps)
        because additional fake voxels have been added
        -> set each voxel’s transparency equal to its value.       
        Parameters:
            cube: 3d data
            title: title for figure.
            init_angle: angle for image plot (from 0-360).
            make_gif: if True create gif from every 5th frames from 3d image plot.
            path_to_save: path to save GIF file.
            """

 
        if self.normalizing:
            cube = self._normalize(cube)
            
        facecolors = cm.gist_stern(cube)          
        facecolors[:,:,:,-1] = cube
        facecolors = self._explode(facecolors)

        filled = facecolors[:,:,:,-1] != 0
        x, y, z = self._expand_coordinates(np.indices(np.array(filled.shape) + 1))

        with plt.style.context("dark_background"):

            fig = plt.figure(figsize=self.figsize)
            ax = fig.gca(projection='3d')

            ax.view_init(30, init_angle)
            ax.set_xlim(right = self.img_dim[0] * 2)
            ax.set_ylim(top = self.img_dim[1] * 2)
            ax.set_zlim(top = self.img_dim[2] * 2)
            ax.set_title(title, fontsize=18, y=1.05)

            ax.voxels(x, y, z, filled, facecolors=facecolors, shade=False)

            if make_gif:
                images = []
                for angle in tqdm(range(0, 360, 5)):
                    ax.view_init(30, angle)
                    fname = str(angle) + '.png'

                    plt.savefig(fname, dpi=120, format='png', bbox_inches='tight')
                    images.append(imageio.imread(fname))
                    #os.remove(fname)
                imageio.mimsave(path_to_save, images)
                plt.close()

            else:
                plt.show()

In [4]:
tensor = np.zeros((512, 512, len(path_to_slices)))
for i in range(len(path_to_slices)):
    image = pydicom.read_file(path_to_slices[i]).pixel_array
    tensor[:,:,i] = image

In [5]:
title = sample_id.replace(".", "/").split("/")[-1]
filename = title+"_3d.gif"

data_to_3dgif = Image3dToGIF3d(img_dim = (120, 120, 78))
transformed_data = data_to_3dgif.get_transformed_data(np.moveaxis(np.flipud(tensor), [0, 1, 2], [-1, -2, -3]))
data_to_3dgif.plot_cube(
    transformed_data[:77, :100, :55],
    title=title,
    make_gif=True,
    path_to_save=filename
)

show_gif(filename, format='png')

(120, 120, 78)


  6%|▌         | 4/72 [07:23<2:05:46, 110.98s/it]


KeyboardInterrupt: 

In [ ]:
end = time.time()

print(end - start)